# SPRINT - 3

## Item - Desenvolvimento do processamento dos dados para geração dos indicadores no Pipeline, e gravação na "Consumer Data"

## Item - Desenvolvimento do processamento dos dados para geração da consolidação de dados da produção no Pipeline, e gravação na "Consumer Data"

<br>
<br>

Autor.: Sérgio C. Medina

#### Declaração dos Pacotes, Libs ou Classes utilizadas no processo.

In [1]:
# Declaração dos Pacotes, Libs ou Classes utilizadas no processo.
import os
import io
import math
import pandas as pd
import gcsfs
import pyarrow
import pyarrow.parquet as pq
from google.cloud import storage
from datetime import datetime, timedelta

import sys
sys.path.append('../../pods')

# configurando variavel de ambiente com o arquivo de credenciais para conexão GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "..\..\secrets\edc-igti-smedina-4920e12ac565.json"
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/sergiomedina/Downloads/edc-igti-smedina-4920e12ac565.json"

# Funções de integração com o Cloud Storage
from utilGCS import utilGCS
gcs = utilGCS(bucketName='edc-pa-i4-data')

# Funções diversas de manipulação de dados
from utilFuncs import utilFuncs
func = utilFuncs()


#### Dados na "processing-zone" pasta "DATAOP"

In [10]:
dtexec = "2021-11-08"
source = "dataop"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataop = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataop = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])

df_dataop['DTPROD'] = df_dataop['DTPROD'].astype(str)


In [11]:
df_dataop

,OP,DTINI,DTFIM,CODMAT,LOTEFAB,QTDPLAN,DTPROD
0,211108L101,2021-11-08 06:00:00,2021-11-09 05:59:59,TB70PVC,TB70394,1320,2021-11-08


#### Dados na "raw-data-zone" pasta "DATACONFIRM"

In [12]:
dtexec = "2021-11-08"
source = "dataconfirm"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataconfirm = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataconfirm = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])

df_dataconfirm['DTPROD'] = df_dataconfirm['DTPROD'].astype(str)

In [6]:
df_dataconfirm

,IDTURNO,DTAPONT,LOTE,PACKID,UNIDADES,KGPACK,KGUNMED,DTPROD
0,1,2021-11-08 08:04:06,TB70394,1,51,295.545,5.795,2021-11-08
1,1,2021-11-08 08:52:20,TB70394,2,49,284.004,5.796,2021-11-08
2,1,2021-11-08 09:42:48,TB70394,3,53,307.188,5.796,2021-11-08
3,1,2021-11-08 10:43:03,TB70394,4,51,295.545,5.795,2021-11-08
4,1,2021-11-08 11:24:04,TB70394,5,50,289.800,5.796,2021-11-08
5,1,2021-11-08 12:21:19,TB70394,6,56,324.576,5.796,2021-11-08
6,1,2021-11-08 13:14:00,TB70394,7,51,295.596,5.796,2021-11-08
7,2,2021-11-08 14:00:46,TB70394,8,53,307.135,5.795,2021-11-08
8,2,2021-11-08 15:02:12,TB70394,9,56,324.576,5.796,2021-11-08
9,2,2021-11-08 15:49:20,TB70394,10,50,289.800,5.796,2021-11-08


In [13]:
df_dataconfirm.groupby(
    ['DTPROD', 'IDTURNO', 'LOTE']
).agg(
    {
        'PACKID': 'count',
        'UNIDADES': 'sum',
        'KGPACK': 'sum',
        'KGUNMED': 'mean'
    }
)


PACKID  UNIDADES    KGPACK   KGUNMED
DTPROD     IDTURNO LOTE                                         
2021-11-08 1       TB70394       7       361  2092.254  5.795714
           2       TB70394       9       488  2828.286  5.795667
           3       TB70394       9       482  2793.515  5.795667

In [16]:
df_dataconfirm.groupby(
    ['DTPROD', 'LOTE']
).agg(
    {
        'PACKID': 'count',
        'UNIDADES': 'sum',
        'KGPACK': 'sum',
        'KGUNMED': 'mean'
    }
)

,,PACKID,UNIDADES,KGPACK,KGUNMED
DTPROD,LOTE,,,,
2021-11-08,TB70394,25,1331,7714.055,5.79568


#### Dados na "raw-data-zone" pasta "DATAPROD"

In [23]:
dtexec = "2021-11-08"
lineid = "101"
source = "dataprod"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataprod = gcs.read_parquet_to_pandas(path=folder)
df_dataprod = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec),('LINE', '=', lineid)])

df_dataprod['DTPROD'] = df_dataprod['DTPROD'].astype(str)
df_dataprod['LINE'] = df_dataprod['LINE'].astype(str)

In [ ]:
df_dataprod.to_csv(path_or_buf='/Users/ych885/Downloads/2021-11-08DL.csv', sep=';', index=False )

In [25]:
#df_dataprod = df_dataprod.loc[(df_dataprod.DTPROD==dtexec) & (df_dataprod.LINE==lineid)]
df_dataprod

,IDTURNO,TIMESTAMP,OP,BATCH,TIMER,TOTMIN,STSID,STSDS,PC,GOOD,REJECT,DTPROD,LINE
0,1,2021-11-08 06:10:46,211108L101,TB70394,00:11:32,11.53,1,SETUP,11.0,0.0,11.0,2021-11-08,101
1,1,2021-11-08 06:13:55,211108L101,TB70394,00:03:09,3.15,1,SETUP,3.0,0.0,3.0,2021-11-08,101
2,1,2021-11-08 06:20:37,211108L101,TB70394,00:06:42,6.70,2,STARTUP,6.0,0.0,6.0,2021-11-08,101
3,1,2021-11-08 06:33:28,211108L101,TB70394,00:12:51,12.85,2,STARTUP,12.0,0.0,12.0,2021-11-08,101
4,1,2021-11-08 06:39:18,211108L101,TB70394,00:05:50,5.83,2,STARTUP,5.0,0.0,5.0,2021-11-08,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,3,2021-11-09 05:30:53,211108L101,TB70394,00:14:14,14.23,4,WORKING,14.0,14.0,0.0,2021-11-08,101
192,3,2021-11-09 05:39:13,211108L101,TB70394,00:08:21,8.35,4,WORKING,8.0,8.0,0.0,2021-11-08,101
193,3,2021-11-09 05:44:24,211108L101,TB70394,00:05:10,5.17,4,WORKING,5.0,5.0,0.0,2021-11-08,101
194,3,2021-11-09 05:51:32,211108L101,TB70394,00:07:09,7.15,4,WORKING,7.0,7.0,0.0,2021-11-08,101


In [ ]:
df_dataprod.dtypes

In [27]:
# Totalizando por Data, turno, Linha e Status
df_sts = pd.DataFrame(df_dataprod.groupby(
#    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH', 'STSID', 'STSDS']
    ['DTPROD', 'IDTURNO', 'LINE', 'STSID', 'STSDS']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum'
    }
)).reset_index()


# Clonando a base carregada dno DATAPROD
df = df_dataprod.copy()

# Preparando para consolidação de dados por Data, turno e Linha 
df['TMIN'] = df['TIMESTAMP']
df['TMAX'] = df['TIMESTAMP']
df.drop(['TIMESTAMP'], axis=1, inplace=True)

df = pd.DataFrame(df.groupby(
    # ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH']
    ['DTPROD', 'IDTURNO', 'LINE']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum',
        'TMIN': 'min',
        'TMAX': 'max'
    }
)).reset_index()


# Tratamento das informações
df['TMAXLIMIT'] = df.apply(lambda row:func.end_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMAXLIMIT'] = pd.to_datetime(df['TMAXLIMIT'])
df['TMAXDIFF'] = (df['TMAXLIMIT']-df['TMAX']).dt.seconds/60
df['WIP'] = df.apply(lambda row:func.round_down(row['TMAXDIFF'], 0),axis=1)

df['TMINLIMIT'] = df.apply(lambda row:func.start_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMINLIMIT'] = pd.to_datetime(df['TMINLIMIT'])
df['TMINDIFF'] = (df['TMIN']-df['TMINLIMIT']).dt.seconds/60

df['DIFF'] = df['TMAXDIFF'].diff().fillna(0)

df['TOTMINADJUSTED'] = round(df['TOTMIN'] + df['DIFF'],0)

df.drop(['TMAX', 'TMAXLIMIT', 'TMAXDIFF'], axis=1, inplace=True)
df.drop(['TMIN', 'TMINLIMIT', 'TMINDIFF', 'DIFF'], axis=1, inplace=True)

# Tot. Tempo turno
df['TPTOTAL'] = 480

print('DEBUG -> [ANTES AJUSTE]:','TOTMINADJUSTED=',df['TOTMINADJUSTED'].sum(),'\n\r',df['TOTMINADJUSTED'])

df['TOTMINADJUSTED'] = df.apply(lambda row:(row['TOTMINADJUSTED'] if row['TOTMINADJUSTED']<480 else 480),axis=1)

print('DEBUG -> [DEPOIS AJUSTE]:','TOTMINADJUSTED=',df['TOTMINADJUSTED'].sum(),'\n\r',df['TOTMINADJUSTED'])

# Tot. Tempo = WORKING
df['TPWORKING'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==4) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

print('DEBUG -> [ANTES AJUSTE]:','TPWORKING=',df['TPWORKING'].sum(),'\n\r',df['TPWORKING'])

df['TPWORKING'] = df.apply(lambda row:(row['TPWORKING'] if row['TPWORKING']<=row['TOTMINADJUSTED'] else row['TOTMINADJUSTED']),axis=1)

print('DEBUG -> [DEPOIS AJUSTE]:','TPWORKING=',df['TPWORKING'].sum(),'\n\r',df['TPWORKING'])

# Tot. Tempo = TPSTOPPLAN
df['TPSTOPPLAN'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

# Tot. Tempo = TPNOALLOC
df['TPNOALLOC'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==0) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)    


# Tempo Programado para produzir
df['TPPROG'] = df['TPTOTAL'] - (df['TPNOALLOC']+df['TPSTOPPLAN'])


# Temp Ociosidade
df['TPIDLE'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID>=1) &
    (df_sts.STSID<3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

df['TPIDLE'] = ((df['TPPROG']-(df['TPWORKING'] + df['TPIDLE']))+df['TPIDLE'])

# Unidades - Produção Teórica
df['PCTHEOR'] = df['TPWORKING'] * 1

# Unidades - Perda por performance
df['PCLOSS'] = df['PCTHEOR'] - df['PC']

# OEE Calc 
df['OEEDISP'] = round(df['TPWORKING'] / df['TPPROG'],3)
df['OEEPERF'] = round(df['PC'] / df['PCTHEOR'],3)
df['OEEQUAL'] = round(df['GOOD'] / df['PC'],3)
df['OEE'] = round(df['OEEDISP'] * df['OEEPERF'] * df['OEEQUAL'],3)

DEBUG -> [ANTES AJUSTE]: TOTMINADJUSTED= 1435.0 
 0    475.0
1    480.0
2    480.0
Name: TOTMINADJUSTED, dtype: float64
DEBUG -> [DEPOIS AJUSTE]: TOTMINADJUSTED= 1435.0 
 0    475.0
1    480.0
2    480.0
Name: TOTMINADJUSTED, dtype: float64
DEBUG -> [ANTES AJUSTE]: TPWORKING= 1388.23 
 0    422.47
1    479.83
2    485.93
Name: TPWORKING, dtype: float64
DEBUG -> [DEPOIS AJUSTE]: TPWORKING= 1382.3 
 0    422.47
1    479.83
2    480.00
Name: TPWORKING, dtype: float64


In [28]:
df

,DTPROD,IDTURNO,LINE,TOTMIN,PC,GOOD,REJECT,WIP,TOTMINADJUSTED,TPTOTAL,...,TPSTOPPLAN,TPNOALLOC,TPPROG,TPIDLE,PCTHEOR,PCLOSS,OEEDISP,OEEPERF,OEEQUAL,OEE
0,2021-11-08,1,101,474.87,444.0,394.0,50.0,5.0,475.0,480,...,0.0,0.0,480.0,57.53,422.47,-21.53,0.88,1.051,0.887,0.820
1,2021-11-08,2,101,479.83,452.0,450.0,2.0,6.0,480.0,480,...,0.0,0.0,480.0,0.17,479.83,27.83,1.00,0.942,0.996,0.938
2,2021-11-08,3,101,485.93,456.0,453.0,3.0,0.0,480.0,480,...,0.0,0.0,480.0,0.00,480.00,24.00,1.00,0.950,0.993,0.943


In [30]:
#df[['DTPROD','IDTURNO','LINE','OP','BATCH','TPTOTAL','TPPROG','TPWORKING','TOTMINADJUSTED','TPSTOPPLAN','TPNOALLOC','TPIDLE']]

#df[['DTPROD','IDTURNO','PC','GOOD','REJECT','WIP','PCTHEOR','PCLOSS']]
df[['OEEDISP', 'TPWORKING', 'TPPROG', 'OEEPERF', 'OEEQUAL', 'OEE']]

        

,OEEDISP,TPWORKING,TPPROG,OEEPERF,OEEQUAL,OEE
0,0.88,422.47,480.0,1.051,0.887,0.820
1,1.00,479.83,480.0,0.942,0.996,0.938
2,1.00,480.00,480.0,0.950,0.993,0.943


In [31]:

df_oee = pd.DataFrame(
    df.groupby(
        ['DTPROD']
    ).agg(
        {
            'TPTOTAL': 'sum',
            'TPPROG': 'sum',
            'TPWORKING': 'sum',
            'TPSTOPPLAN': 'sum',
            'TPNOALLOC': 'sum',
            'TPIDLE': 'sum',
            'PC': 'sum',
            'GOOD': 'sum',
            'REJECT': 'sum',
            'WIP': 'sum',
            'PCTHEOR': 'sum',
            'PCLOSS': 'sum',

        }
    )
).reset_index()

df_oee['OEEDISP'] = round(df_oee['TPWORKING']/df_oee['TPPROG'],3)

df_oee['OEEPERF'] = round(df_oee['PC'] / df_oee['PCTHEOR'],3)

df_oee['OEEQUAL'] = round(df_oee['GOOD'] / df_oee['PC'],3)

df_oee['OEE'] = round(df_oee['OEEDISP'] * df_oee['OEEPERF'] * df_oee['OEEQUAL'],3)


df_oee


,DTPROD,TPTOTAL,TPPROG,TPWORKING,TPSTOPPLAN,TPNOALLOC,TPIDLE,PC,GOOD,REJECT,WIP,PCTHEOR,PCLOSS,OEEDISP,OEEPERF,OEEQUAL,OEE
0,2021-11-08,1440,1440.0,1382.3,0.0,0.0,57.7,1352.0,1297.0,55.0,11.0,1382.3,30.3,0.96,0.978,0.959,0.9


In [32]:
# df.agg(
#     {
#         'OEEDISP': 'mean',
#         'OEEPERF': 'mean',
#         'OEEQUAL': 'mean',
#         'OEE': 'mean'
#     }
# )


df.groupby(
    ['DTPROD']
).agg(
    {
        'OEEDISP': 'mean',
        'OEEPERF': 'mean',
        'OEEQUAL': 'mean',
        'OEE': 'mean'
    }
)




,OEEDISP,OEEPERF,OEEQUAL,OEE
DTPROD,,,,
2021-11-08,0.96,0.981,0.958667,0.900333
